In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('COL774_A5_SVM/train_ls.csv')
X = data.drop('target', axis=1).values
Y = data['target'].values
Y = np.where(Y==0,-1,1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
n, d = X.shape

In [2]:
w = cp.Variable(d)        
b = cp.Variable()        
slack = cp.Variable(n)   

C = 1  
objective = cp.Minimize(cp.norm1(w)/2 + C * cp.sum(slack))

constraints = []
for i in range(n):
    constraints.append(Y[i] * (X[i] @ w + b) >= 1 - slack[i]) 
    constraints.append(slack[i] >= 0) 

problem = cp.Problem(objective, constraints)
problem.solve()

print(f"Optimal weights: {w.value}")
print(f"Optimal bias: {b.value}")
print(max(slack.value))

Optimal weights: [ 1.08733809e-11 -2.54047555e-01  6.41014474e-01  9.49696327e-01]
Optimal bias: 0.08844444448521421
2.4542817472452488e-12


In [3]:
support_vectors = []
for i in range(n):
    margin_distance = Y[i] * (X[i] @ w.value + b.value)
    if np.isclose(margin_distance, 1, atol=1e-4):  
        support_vectors.append(i)

print(f"Support vectors (indices): {support_vectors}")
print(f"Number of support vectors: {len(support_vectors)}")

Support vectors (indices): [63, 70, 71, 74]
Number of support vectors: 4
